In [1]:
# Import all the important libraries
import numpy as np
import numba as nb 
import matplotlib.pyplot as plt
from matplotlib.cm import ScalarMappable
import matplotlib

%matplotlib inline
from ipywidgets import interact

from sys_pde_no_growth_no_Lmx1a import *

In [2]:
# System Params - Same for all the species
sys_params = np.zeros(2)
sys_params[0] = 2 # Hill coefficient
sys_params[1] = 0.7 # BMP(t=0)

# BMP
b_params = np.zeros(4)
b_params[0] = 1 # D_b
b_params[1] = 1 # a_b_s
b_params[2] = 0.1 # K_b_s
b_params[3] = 0.2 # d_b

# BMP i
i_params = np.zeros(4)
i_params[0] = 1.5 # D_i
i_params[1] = 1 # a_i_s
i_params[2] = 0.1 # K_i_s
i_params[3] = 0.1 # d_i

# pSmad
s_params = np.zeros(4)
s_params[0] = 1 # a_s_b
s_params[1] = 0.1 # K_s_b
s_params[2] = 0.2 # K_s_i
s_params[3] = 1 # d_s

In [3]:
# Define all the simulations params
sol_params = np.zeros(10)
sol_params[0] = 0.05 # dt 
sol_params[1] = 1 # dx
sol_params[2] = 250/sol_params[1] #Lx
sol_params[3] = 250/sol_params[1] #Ly
sol_params[4] = 100/sol_params[1] # Radius of cell colony
sol_params[5] = 160/sol_params[0] # Total time, T
sol_params[6] = 2/sol_params[0] # Delta t (snapshot saving time)
sol_params[7] = 8/sol_params[0] # Cell Colony starts growing then (early migratory cells)
sol_params[8] = 0.25*sol_params[0] # Cell colony growth rate

Run for params and save figs

In [4]:
num = 0
for a_b_s in [0.1, 0.25, 0.5, 0.75, 1, 1.25, 1.5, 1.75, 2]:
    for a_i_s in [0.1, 0.25, 0.5, 0.75, 1, 1.25, 1.5, 1.75, 2]:
        for a_s_b in [0.1, 0.25, 0.5, 0.75, 1, 1.25, 1.5, 1.75, 2]:
            for K_b_s in [0.01, 0.05, 0.1, 0.2, 0.5]:
                for K_i_s in [0.01, 0.05, 0.1, 0.2, 0.5]:
                    for K_s_b in [0.01, 0.05, 0.1, 0.2, 0.5]:
                        for K_s_i in [0.01, 0.05, 0.1, 0.2, 0.5]:
                            num += 1
                            print(num)
                            b_params[1] = a_b_s
                            b_params[2] = K_b_s

                            i_params[1] = a_i_s 
                            i_params[2] = K_i_s

                            s_params[0] = 1
                            s_params[1] = 0.1
                            s_params[2] = 0.2

                            # Get the initial conditions
                            array_b, array_i, array_s = create_IC_1(sys_params, sol_params)

                            data_b_time, data_i_time, data_s_time = run_and_save_sys(array_b, array_i, 
                                                array_s, b_params, i_params, s_params,
                                                sys_params, sol_params)
                            
                            # Plot all the concs as a function of space over one axis from the center
                            fig, axes = plt.subplots(2, 2, figsize=(8, 8), dpi=300)
                            Lx, Ly = int(sol_params[2]), int(sol_params[3])
                            R = int(sol_params[4])
                            Tlen = sol_params[5]/sol_params[6]

                            T_points = np.array([0, 1, 2, 6, 12, 24, 48])
                            cmap_vals = [0.1, 0.3, 0.45, 0.6, 0.8, 0.9, 1]

                            norm = matplotlib.colors.Normalize(vmin=0, vmax=1) 
                            cmap1 = plt.cm.Blues
                            cmap2 = plt.cm.Greens
                            cmap3 = plt.cm.Purples
                            cmap4 = plt.cm.Reds


                            for i, t in enumerate(T_points):
                                b_concs_slice_x = data_b_time[t][(Lx//2+R-20):(Lx//2+R+20), 
                                                                 Ly//2]/np.max(data_b_time)
                                i_concs_slice_x = data_i_time[t][(Lx//2+R-20):(Lx//2+R+20), 
                                                                 Ly//2]/np.max(data_i_time)
                                s_concs_slice_x = data_s_time[t][(Lx//2+R-20):(Lx//2+R+20), 
                                                                 Ly//2]/np.max(data_s_time)


                                axes[0,0].plot(np.arange(40) - 20, b_concs_slice_x, 
                                               color = cmap1(norm(cmap_vals[i]))
                                               , label="{}h".format(sol_params[6]*sol_params[0]*i))
                                axes[0,0].set_title("BMP")
                                axes[1,0].plot(np.arange(40) - 20, i_concs_slice_x, 
                                               color = cmap2(norm(cmap_vals[i]))
                                               , label="{}h".format(sol_params[6]*sol_params[0]*i))
                                axes[1,0].set_title("BMPi")
                                axes[0,1].plot(np.arange(40) - 20, s_concs_slice_x, 
                                               color = cmap3(norm(cmap_vals[i]))
                                               , label="{}h".format(sol_params[6]*sol_params[0]*i))
                                axes[0,1].set_title("pSmad")

                            axes[0,0].legend()
                            axes[1,0].legend()
                            axes[0,1].legend()
                            fig.supylabel(r'Concentrations $c/c_{max}$')
                            fig.supxlabel('distance from edge (dx)')
                            plt.suptitle("Concentration profiles for a slice of system at different times")
                            plt.savefig("NoGrowth/a_bs_{}_a_is_{}_a_sb_{}_K_bs_{}_K_is_{}_K_sb_{}_K_si_{}.jpeg".
                                        format(a_b_s, a_i_s, a_s_b, K_b_s, K_i_s, K_s_b, K_s_i))
                            plt.close()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25


SystemError: CPUDispatcher(<function solve_sys_pde_lhs at 0x7056b79b7490>) returned a result with an exception set